In [18]:
import itertools
import numpy as np

class SimplexComplex:
    def __init__(self, simplices):
        self.simplices = simplices
        self.face_set = self.calculate_face_set()

    def add_simplex(self, simplex):
        if simplex not in self.simplices:
            self.simplices.append(simplex)
            self.face_set = self.calculate_face_set()

    def remove_simplex(self, simplex):
        if simplex in self.simplices:
            self.simplices.remove(simplex)
            self.face_set = self.calculate_face_set()

    def dimension(self):
        if not self.simplices:
            return -1
        return max(len(simplex) - 1 for simplex in self.simplices)

    def calculate_face_set(self):
        all_faces = set()
        for simplex in self.simplices:
            for i in range(1, len(simplex) + 1):
                for combo in itertools.combinations(simplex, i):
                    all_faces.add(tuple(sorted(combo)))
        return all_faces

    def n_faces(self, target_dimension):
        return set(face for face in self.face_set if len(face) == target_dimension + 1)
  
    def st(self,simplex):
        star = set()
        star.add(simplex)
        for k in range((len(simplex)-1) + 1, self.dimension() + 1):
            star.update(face for face in self.n_faces(k) if set(simplex).issubset(face))
        return star
    
    def faces_simplex(self, simplex):
        lower_dimension_faces = set()
        simplex_set = set(simplex)

        for k in range(0, (len(simplex)-1) + 1):
            for face in self.n_faces(k):
                if set(face).issubset(simplex_set):
                    lower_dimension_faces.add(face)

        return lower_dimension_faces
    
    def faces_simplex_codimension_1(self, simplex):
        lower_dimension_faces = set()
        simplex_set = set(simplex)

        for k in range(0, (len(simplex)-1) + 1):
            for face in self.n_faces(k):
                if set(face).issubset(simplex_set) and abs((len(face)-1) - (len(simplex)-1))==1:
                    lower_dimension_faces.add(face)

        return lower_dimension_faces
    
    def Euler_characteristic(self):
        n = self.dimension()
        euler_sum = sum(((-1)**k) * len(self.n_faces(k)) for k in range(n + 1))
        return euler_sum
    
    def closed_star(self, simplex):
        # Calcula la estrella cerrada de la cara dada
        star_closed_faces = self.st(simplex)

        # Inicializa un nuevo complejo simplicial con las caras de la estrella cerrada
        closed_star_complex = SimplexComplex(star_closed_faces)

        # Devuelve las caras del nuevo complejo simplicial
        return closed_star_complex.face_set

    def lk(self, face):
        # Calcula el link de la cara dada
        link_faces = list(filter(lambda x: len(set(x).intersection(face)) == 0, self.closed_star(face)))

        # Devuelve el link de la cara entre llaves
        return "{" + ", ".join(map(str, link_faces)) + "}"
    
    def connected_components(self):
        visited = set()
        components = []

        def dfs(vertex, component):
            visited.add(vertex)
            component.add(vertex)

            for neighbor in neighbors[vertex]:
                if neighbor not in visited:
                    dfs(neighbor, component)

        neighbors = {vertex: set() for simplex in self.simplices for vertex in simplex}

        for simplex in self.simplices:
            for vertex in simplex:
                neighbors[vertex].update(set(simplex) - {vertex})


        for vertex in neighbors:
            if vertex not in visited:
                new_component = set()
                dfs(vertex, new_component)
                components.append(new_component)

        return len(components)


    def insert(self, new_simplices,value=0.0):
        for simplex in new_simplices:
            self.add_simplex(simplex)
    
    def boundarymatrix(self, dimension):
        if dimension == 0:
            matrix = np.zeros((1, len(self.n_faces(dimension))), dtype=int)
            return matrix
        else:
            simplices_dim = list(self.n_faces(dimension))
            simplices_dim_minus_1 = list(self.n_faces(dimension - 1))

            # Ordenar lexicográficamente las listas
            simplices_dim.sort()
            simplices_dim_minus_1.sort()
    
            matrix = np.zeros((len(simplices_dim_minus_1), len(simplices_dim)), dtype=int)
            
            for i, simplex_dim_minus_1 in enumerate(simplices_dim_minus_1):
                for j, simplex_dim in enumerate(simplices_dim):
                    if simplex_dim in self.st(simplex_dim_minus_1):
                        matrix[i, j] = 1

            return matrix
        
    def general_boundarymatrix(self):
        dimension = self.dimension()
        if dimension == 0:
            matrix = np.zeros((1, len(self.n_faces(dimension))), dtype=int)
            return matrix
        else:
            simplices_dim = list(self.face_set)
    
            matrix = np.zeros((len(simplices_dim), len(simplices_dim)), dtype=int)
            
            for i, simplex_dim in enumerate(simplices_dim):
                for j, simplex_dim in enumerate(simplices_dim):
                    if simplex_dim in self.faces_simplex_codimension_1(simplex_dim):
                        matrix[i, j] = 1

            return matrix
    
    def smith_normal_form(self,matrix):
        matrix = np.array(matrix)
        m, n = matrix.shape
        row_permutation_matrix = np.eye(m)
        col_permutation_matrix = np.eye(n)

        for i in range(min(m, n)):
            non_zero_indices = np.where(matrix[i:, i:] != 0)
            if len(non_zero_indices[0]) == 0:
                continue

            first_non_zero_row = non_zero_indices[0][0] + i
            first_non_zero_col = non_zero_indices[1][0] + i

            matrix[[i, first_non_zero_row]] = matrix[[first_non_zero_row, i]]
            row_permutation_matrix[[i, first_non_zero_row]] = row_permutation_matrix[[first_non_zero_row, i]]
            matrix[:, [i, first_non_zero_col]] = matrix[:, [first_non_zero_col, i]]
            col_permutation_matrix[:, [i, first_non_zero_col]] = col_permutation_matrix[:, [first_non_zero_col, i]]

            for j in range(i + 1, m):
                if matrix[j, i] != 0:
                    matrix[j] = (matrix[j] + matrix[i]) % 2
            for k in range(i + 1, n):
                if matrix[i, k] != 0:
                    matrix[:, k] = (matrix[:, k] + matrix[:, i]) % 2

        return matrix

    def Betti_number(self,dimension):
        matrix_boundary=self.boundarymatrix(dimension)
        matrix_smith=self.smith_normal_form(matrix_boundary)
        num_rows, num_cols = matrix_smith.shape
        null_columns = np.all(matrix_smith == 0, axis=0)
        non_null_columns = np.any(matrix_smith != 0, axis=0)

        num_null_columns = np.sum(null_columns)
        num_non_null_columns = np.sum(non_null_columns)

        #siguiente dimension

        matrix_boundary_next=self.boundarymatrix(dimension+1)
        matrix_smith_next=self.smith_normal_form(matrix_boundary_next)
        num_rows_next, num_cols_next = matrix_smith_next.shape
        null_columns_next = np.all(matrix_smith_next == 0, axis=0)
        non_null_columns_next = np.any(matrix_smith_next != 0, axis=0)

        num_null_columns_next = np.sum(null_columns_next)
        num_non_null_columns_next = np.sum(non_null_columns_next)

        return num_null_columns - num_non_null_columns_next 
    
    def algoritmo_incremental(self):
        betti0 = 0
        betti1 = 0
        face_set_list = sorted(list(self.face_set), key=lambda simplex: (len(simplex), simplex))
        #print("faces",face_set_list)
        for i in range(self.dimension()+1):
            for idx, (simplex) in enumerate(face_set_list):
                N_i = SimplexComplex(face_set_list[:idx+1])
                #print(N_i)
                N_i_menos_1 = SimplexComplex(face_set_list[:idx])
                #print(N_i_menos_1)
                if i == (len(simplex) - 1):
                    if i == 0:
                        betti0 = betti0 + 1
                    elif i == 2:
                        betti1 = betti1 - 1
                    elif i == 1:
                        if N_i.connected_components() == N_i_menos_1.connected_components():
                            #print(N_i.connected_components())
                            betti1 = betti1 + 1
                        else:
                            betti0 = betti0 -1
        return betti0, betti1



    def __str__(self):
        return str(self.simplices)

# Ejemplo de uso
print("Practica 1")
sc = SimplexComplex([(0, 1, 2, 3)])
print("Complejo simplicial:", sc)
print("Dimension del complejo simplicial:", sc.dimension())
print("Conjunto de todas las caras:", sc.face_set)
print("Conjunto de caras de dimensión especifica:", sc.n_faces(3))
print("Estrella del símplice", sc.st((0,)))
print("Link del simplice", sc.lk((0,1)))
print("Característica de Euler:", sc.Euler_characteristic())
print("Número de componentes conexas:", sc.connected_components())

print("Caras de un simplice",sc.faces_simplex((0,1,3)))
print("Caras codimension 1 de un simplice",sc.faces_simplex_codimension_1((0,1,3)))




print("\nPractica 3")

boundary_matrix = sc.boundarymatrix(1)
print("\nCalculo de matriz de borde de dimension dada:")
print(boundary_matrix)
result_matrix = sc.smith_normal_form(boundary_matrix)
print("\nMatriz en forma normal de Smith:")
print(result_matrix)


print("Numeros de Betti del:")
print("Tetraedro:")
print(sc.Betti_number(0),sc.Betti_number(1),sc.Betti_number(2),sc.Betti_number(3))
print("Borde del tetraedro:")
sc1=SimplexComplex(list(sc.n_faces(2)))
print(sc1.Betti_number(0),sc1.Betti_number(1),sc1.Betti_number(2),sc1.Betti_number(3))
print("El toro con las dos triangulaciones vistas en clase:")
sc2_1=SimplexComplex([(1,2,4),(2,4,5),(2,3,5),(3,5,6),(1,3,6),(1,4,6),(4,5,7),(5,7,8),(5,6,8),(6,8,9),(4,6,9),(4,7,9),(1,7,8),(1,2,8),(2,8,9),(2,3,9),(3,7,9),(1,3,7)])
print(sc2_1.Betti_number(0),sc2_1.Betti_number(1),sc2_1.Betti_number(2),sc2_1.Betti_number(3))
sc2_2=SimplexComplex([(1,7,3),(1,2,3),(2,5,7),(3,2,6),(2,5,6),(3,5,7),(3,4,6),(6,5,1),(1,4,5),(3,4,5),(4,7,6),(1,6,7),(1,2,4),(2,4,7)])
print(sc2_2.Betti_number(0),sc2_2.Betti_number(1),sc2_2.Betti_number(2),sc2_2.Betti_number(3))
print("El plano proyectivo:")
sc3=SimplexComplex([(1,2,6),(2,3,4),(1,3,4),(1,2,5),(2,3,5),(1,3,6),(2,4,6),(1,4,5),(3,5,6),(4,5,6)])
print(sc3.Betti_number(0),sc3.Betti_number(1),sc3.Betti_number(2),sc3.Betti_number(3))
print("La botella de Klein:")
sc4=SimplexComplex([(0,1,3),(1,3,4),(1,2,4),(2,4,5),(2,5,0),(0,5,6),(3,4,6),(6,4,7),(4,5,7),(5,7,8),(8,5,6),(3,6,8),(0,7,6),(1,0,7),(1,7,8),(1,2,8),(2,3,8),(2,3,0)])
print(sc4.Betti_number(0),sc4.Betti_number(1),sc4.Betti_number(2),sc4.Betti_number(3))
print("El anillo:")
sc5=SimplexComplex([(1,2,4),(1,3,6),(1,4,6),(2,3,5),(2,4,5),(3,5,6)])
print(sc5.Betti_number(0),sc5.Betti_number(1),sc5.Betti_number(2),sc5.Betti_number(3))
print("El sombrero del asno:")
sc6=SimplexComplex([(1,2,4),(1,3,4),(3,4,8),(3,2,8),(1,2,8),(2,4,5),(5,6,4),(6,4,8),(8,6,7),(1,7,8),(1,2,7),(3,1,5),(1,5,6),(1,3,6),(6,3,7),(2,7,3)])
print(sc6.Betti_number(0),sc6.Betti_number(1),sc6.Betti_number(2),sc6.Betti_number(3))
print("Del complejo simplicial de la transparencia 4 del documento Homología Simplicial II:")
sc7=SimplexComplex([(0,1),(1,2,3,4),(4,5),(5,6),(4,6),(6,7,8),(8,9)])
print(sc7.Betti_number(0),sc7.Betti_number(1),sc7.Betti_number(2),sc7.Betti_number(3))
print("Del doble toro:")
print(sc.Betti_number(0),sc.Betti_number(1),sc.Betti_number(2),sc.Betti_number(3))
print("De algunos alfa complejos:")
print(sc.Betti_number(0),sc.Betti_number(1),sc.Betti_number(2),sc.Betti_number(3))
print("Con el algoritmo incemental:")
sc9=SimplexComplex([(1,2,0),(0,2,9),(2,3,6),(3,4,6),(4,5,6),(5,6,7),(7,9),(7,8),(8,9)])
print(sc9.algoritmo_incremental())

Practica 1
Complejo simplicial: [(0, 1, 2, 3)]
Dimension del complejo simplicial: 3
Conjunto de todas las caras: {(0, 1), (1, 3), (2,), (1, 2), (0, 1, 2), (0, 1, 3), (0, 3), (0, 2, 3), (1, 2, 3), (2, 3), (1,), (0, 2), (0, 1, 2, 3), (0,), (3,)}
Conjunto de caras de dimensión especifica: {(0, 1, 2, 3)}
Estrella del símplice {(0, 1), (0, 1, 2), (0, 1, 3), (0, 3), (0, 2, 3), (0, 2), (0, 1, 2, 3), (0,)}
Link del simplice {(2,), (2, 3), (3,)}
Característica de Euler: 1
Número de componentes conexas: 1
Caras de un simplice {(0, 1), (1, 3), (0, 1, 3), (0, 3), (1,), (0,), (3,)}
Caras codimension 1 de un simplice {(0, 1), (0, 3), (1, 3)}

Practica 3

Calculo de matriz de borde de dimension dada:
[[1 1 1 0 0 0]
 [1 0 0 1 1 0]
 [0 1 0 1 0 1]
 [0 0 1 0 1 1]]

Matriz en forma normal de Smith:
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 0 0]]
Numeros de Betti del:
Tetraedro:
1 0 0 0
Borde del tetraedro:
1 0 1 0
El toro con las dos triangulaciones vistas en clase:
1 2 1 0
1 2 1 0
El plano p